In [ ]:
""" neural network """
# importing packages
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn import metrics
import torch
import torch.nn as nn 
import torch.nn.functional as F
from torch.autograd import Variable
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader, Dataset
from pickle import dump, load
from scipy.stats import pearsonr
from sklearn.decomposition import PCA
import tqdm
import datetime

In [ ]:
# helper functions
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

Original matched catalog had ~29k AGN.  Filtering out errors > .4 dex brings catalog to ~23,000 AGN.  I find that filtering out errors >.3 dex gives best performance.  This brings full catalog to 20,549 AGN.

In [ ]:
# Black Hole dataset--default parameters are for training and predicting AGN mass.  Pass 'train=False' 
# for test-set and 'mass=False' for AGN redshift prediction.
class BHDataset(Dataset):
    def __init__(self, path, train=True, mass=True):
        self.path = path
        self.train = train
        self.mass = mass
        self.sc = StandardScaler()
        self.pca = PCA(.95)
        
        if self.mass:
            
            if self.train:
                self.data = pd.read_csv(self.path + 'TRAIN_dr14.csv')
                self.features = self.sc.fit_transform(np.asarray(self.data.iloc[:,[6,8,9,10,11,12,13,14,15,16,17,18]]))
                self.pca.fit(self.features)
                self.features = self.pca.transform(self.features)
                dump(self.sc, open('train_scaler.pkl','wb'))
                dump(self.pca, open('train_scaler_pca.pkl','wb'))
        
            else:
                self.data = pd.read_csv(self.path + 'TEST_dr14.csv')
                self.sc = load(open('train_scaler.pkl','rb'))
                self.pca = load(open('train_scaler_pca.pkl','rb'))
                self.features = self.sc.transform(np.asarray(self.data.iloc[:,[6,8,9,10,11,12,13,14,15,16,17,18]]))
                self.features = self.pca.transform(self.features)
        else:
            
            if self.train:
                self.data = pd.read_csv(self.path + 'TRAIN_dr14.csv')
                self.features = self.sc.fit_transform(np.asarray(self.data.iloc[:,9:]))
                self.pca.fit(self.features)
                self.features = self.pca.transform(self.features)
                dump(self.sc, open('train_scaler.pkl','wb'))
                dump(self.pca, open('train_scaler_pca.pkl','wb'))
        
            else:
                self.data = pd.read_csv(self.path + 'TEST_dr14.csv')
                self.sc = load(open('train_scaler.pkl','rb'))
                self.pca = load(open('train_scaler_pca.pkl','rb'))
                self.features = self.sc.transform(np.asarray(self.data.iloc[:,9:]))
                self.features=(self.pca.transform(self.features))
            
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, index):
        
        if self.mass:
            
            ID = torch.from_numpy(np.asarray(self.data.iloc[index,0]))
            target = torch.from_numpy(np.asarray(self.data.iloc[index,5]))
            features = torch.from_numpy(self.features[index].reshape(1,-1).squeeze())
            error = torch.from_numpy(np.asarray(self.data.iloc[index,7]))
            return (ID, features, target, error)
        
        else:
            ID = torch.from_numpy(np.asarray(self.data.iloc[index,0]))
            target = torch.from_numpy(np.asarray(self.data.iloc[index,6]))
            features = torch.from_numpy(self.features[index].reshape(1,-1).squeeze())
            return (ID, features, target)

        
# define train and test datasets.  Train test split was done previously using sklearn.
train_mass = BHDataset('../../data/dr14/')
test_mass = BHDataset('../../data/dr14/', train=False)

train_z = BHDataset('../../data/dr14/', mass=False)
test_z = BHDataset('../../data/dr14/', mass=False, train=False)
train_mass.__getitem__(0)

PCA reduces dimensionality of AGN BH Mass prediction from 12 features --> 6 features.  PCA reduces dimensionality of AGN redshift prediction from 10 features --> 5 features.

In [ ]:
# Define dataloaders with the datasets.  Only shuffle training sets.
train_dl_mass = DataLoader(train_mass, batch_size=256, shuffle=True)
test_dl_mass = DataLoader(test_mass, batch_size=256, shuffle=False)

train_dl_z = DataLoader(train_z, batch_size=256, shuffle=True)
test_dl_z = DataLoader(test_z, batch_size=256, shuffle=False)

In [ ]:
# default architecture is to predict AGN mass.  Pass 'mass=False' to predict redshift.
class AGNet(nn.Module):
    def __init__(self, mass=True):
        super().__init__()
        self.mass = mass
        
        if self.mass:

            self.fc1 = nn.Linear(6, 32)
            self.fc2 = nn.Linear(32, 64)
            self.fc3 = nn.Linear(64, 128)
            self.fc4 = nn.Linear(128, 128)
            self.fc5 = nn.Linear(128, 64)
            self.fc6 = nn.Linear(64, 32)
            self.fc7 = nn.Linear(32, 1)
            
        else:
            
            self.fc1 = nn.Linear(5, 32)
            self.fc2 = nn.Linear(32, 64)
            self.fc3 = nn.Linear(64, 128)
            self.fc4 = nn.Linear(128, 128)
            self.fc5 = nn.Linear(128, 64)
            self.fc6 = nn.Linear(64, 32)
            self.fc7 = nn.Linear(32, 1)
            

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        x = F.relu(self.fc5(x))
        x = F.relu(self.fc6(x))
        x = self.fc7(x)
        return x

# defining neural networks
net_mass = AGNet()
net_z = AGNet(mass=False)

In [ ]:
# count parameters for reference.  For both redshift and mass, (training size/model parameters) < 1.
(count_parameters(net_mass), count_parameters(net_z))

In [ ]:
def weighted_mse_loss(mu, target, var):
    return torch.mean(var**-1 * (mu - target) ** 2 + var)

def weighted_smoothl1_loss(mu, target, var):
    if torch.abs(torch.mean(mu) - torch.mean(target)) < 1:
        loss = torch.mean(0.5*(mu - target)^2 + var)
    else:
        loss = torch.mean(torch.abs(mu - target) - 0.5 + var)

In [ ]:
# defining optimizer and loss function
lr = .005 # .01 for redshift
optimizer = optim.AdamW(net_z.parameters(), lr=lr)
loss_function = nn.SmoothL1Loss() #for mass, nn.MSELoss() for redshift

In [ ]:
# training loop takes in epoch #, network, train loader, loss function, and optimizer.  
def train(num_epochs, trainloader, testloader, mdl, batch_size=256):
    
    best_rmse = float('inf')
    epoch_list = np.linspace(1, num_epochs , num = num_epochs)
    train_loss_list, train_rmse_list, test_loss_list, test_rmse_list = list(), list(), list(), list()

    for epoch in range(num_epochs):

            train_pred, train_gt, test_pred, test_gt = list(), list(), list(), list()

            for data in trainloader:

                ID, features, ground_truth= data
                train_gt.append(ground_truth.float())
                output_train = mdl(features.float())
                train_pred.append(output_train.float())
                train_loss = loss_function(output_train.squeeze(), ground_truth.float().squeeze())
                mdl.zero_grad()
                train_loss.backward()
                optimizer.step()
                
            train_loss_list.append(train_loss.item()/batch_size)
            train_ground_truth = torch.cat(train_gt).data
            train_predictions = torch.cat(train_pred).data.flatten()
            train_rmse = np.sqrt(metrics.mean_squared_error(train_ground_truth, train_predictions))
            train_rmse_list.append(train_rmse)
            
            with torch.no_grad():
                
                for data_ in testloader:  

                    ID_, features_, ground_truth_ = data_  
                    test_gt.append(ground_truth_.float())
                    output_test = mdl(features_.float()) 
                    test_pred.append(output_test.float())
                    test_loss = loss_function(output_test.squeeze(), ground_truth_.squeeze()) # was.float() before
                    
                test_loss_list.append(test_loss.item()/batch_size)
                test_ground_truth = torch.cat(test_gt).data.flatten()
                test_predictions = torch.cat(test_pred).data.flatten()
                test_rmse = np.sqrt(metrics.mean_squared_error(test_ground_truth, test_predictions))
                test_rmse_list.append(test_rmse)
                
                if test_rmse < best_rmse:
                    best_rmse = test_rmse
                    datetime_today = str(datetime.date.today())
                    torch.save(net_mass, '../saved_models/' + datetime_today + 'sneh' + str(best_rmse) + 'mlp.mdl')
                    print("saved to " + "sneh_mlp.mdl" + " file.")
                    
                print(f'EPOCH: {epoch+1}\nTrain Loss: {train_loss.item():.5f} | Train RMSE: {train_rmse:.5f}\nTest Loss: {test_loss.item():.5f} | Test RMSE: {test_rmse:.5f}')
            
    return epoch_list, train_loss_list, test_loss_list, train_rmse_list, test_rmse_list

In [ ]:
def plot_loss_curve(epoch_list, train_loss, test_loss, train_rmse, test_rmse):
    fig, (ax1, ax2) = plt.subplots(2, 1)
#     ax1.set_xticks(len(epoch_list)/5)
#     ax2.set_xticks(len(epoch_list)/5)
    fig.set_figheight(10)
    fig.set_figwidth(15)
    
    fig.suptitle('Loss and RMSE Curves', fontsize=16)
    
    ax1.set(xlabel = 'EPOCH', ylabel = 'LOSS')
    ax1.plot(epoch_list, train_loss, label = 'Train Loss', color = 'blue')
    ax1.plot(epoch_list, test_loss, label = 'Test Loss', color = 'orange', ls='--')
    ax1.legend()
    
    ax2.set(xlabel = 'EPOCH', ylabel = 'RMSE')
    ax2.set_ylim([0,1])
    ax2.plot(epoch_list, train_rmse, label = 'Train RMSE', color = 'blue')
    ax2.plot(epoch_list, test_rmse, label = 'Test RMSE', color = 'orange', ls='--') 
    ax2.legend()
    

In [ ]:
# must change train function parameters depending on mass or z training
epoch_list, train_loss, test_loss, train_rmse, test_rmse = train(100, train_dl_z, test_dl_z, net_z)

In [ ]:
# this will return loss vs. epoch and RMSE vs. epoch curves
plot_loss_curve(epoch_list, train_loss, test_loss, train_rmse, test_rmse)    

The current combination of number of epochs, loss function, and optimizer are what we found to work best.  We encourage users to explore other combinations!

In [ ]:
# save model
# torch.save(net_mass.state_dict(), '../../models/AGNet_dr14_6features_lr.005_50epoch.mdl')

In [ ]:
# test loop outputs plot of results + dataframe of object ID, ground truth values, and network predictions
def test(testloader, mdl, df=False):

    with torch.no_grad():
        
        test_ID, test_pred, test_gt = list(), list(), list()

        for data in testloader:
            ID, features, ground_truth = data  
            test_ID.append(ID.float())
            test_gt.append(ground_truth.float())
            output_test = mdl(features.float()) 
            test_pred.append(output_test.float())
            test_loss = loss_function(output_test.squeeze(), ground_truth.squeeze())

        test_ground_truth = torch.cat(test_gt).data
        test_predictions = torch.cat(test_pred).data.flatten()
        test_rmse = np.sqrt(metrics.mean_squared_error(test_ground_truth, test_predictions))
        ID = torch.cat(test_ID).data
        
        plt.figure(figsize=[8,8])
        plt.plot(test_ground_truth, test_ground_truth,color='black', label = 'Mass Ground Truth')
        plt.scatter(test_ground_truth, test_predictions,s=2, color='blue', label = 'NN prediction')
        plt.title('RMSE:' + str(test_rmse) + ' | R2: ' + str(metrics.r2_score(test_ground_truth,test_predictions)) + '| Pearson:' + str(pearsonr(test_ground_truth,test_predictions)[0]))
        plt.xlabel('AGN Mass')
        plt.ylabel('AGN Mass')
#         plt.savefig('/Users/SnehPandya/Desktop/plot.png', facecolor='white')
        plt.legend()
        plt.show()
        
    if df: 
        
        results = pd.DataFrame({'my_ID':ID.numpy().astype('int'), 'ground_truth':test_ground_truth.numpy(), 'network_predictions':test_predictions.numpy() })
        return results
    
    else:
        print('pass df=True to create dataframe')

In [ ]:
results = test(test_dl_mass, net_mass, df=True)